In [5]:
import os
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from subprocess import call

In [6]:
# load CSV files
league = 'spanish'

df = None
files = os.listdir(f'fulldata/{league}')
files.sort()
for file in files:
    year = int(file.strip('.csv'))
    df_year = pd.read_csv(f'fulldata/{league}/' + file)
    df_year['Year'] = year
    df_year['Match'] = df_year.index + 1

    if df is None:
        df = df_year
    else:
        df = df.append(df_year, ignore_index=True, sort=False)

In [7]:
# remove unused columns
df_league = None
df.reset_index(inplace=True)
df = df[['Year', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR']]

predict_year = 2017

teams = np.unique(df.loc[df['Year'] == predict_year, 'HomeTeam'].values)
teams.sort()
for team in teams:

    df_team = df[(df['HomeTeam'] == team) | (df['AwayTeam'] == team)]
    X = pd.DataFrame(
        data={
            'Year': df_team['Year'],
            'Date': df_team['Date'],
            'HomeMatch': df_team['HomeTeam'] == team
        }
    )
    X['Opponent'] = np.where(X['HomeMatch'], df_team['AwayTeam'], df_team['HomeTeam'])
    X['FTR'] = df_team['FTR']
    X['Won'] = np.where(X['HomeMatch'], df_team['FTR'] == 'H', df_team['FTR'] == 'A')
    X['Draw'] = df_team['FTR'] == 'D'
    X['Lost'] = np.where(X['HomeMatch'], df_team['FTR'] == 'A', df_team['FTR'] == 'H')
    X['Result'] = np.where(X['Won'], 'Win', (np.where(X['Lost'], 'Lose', 'Draw')))

    # find number of times won against this opponent in last 5 meetings
    for key, groupByOpponent in X.groupby('Opponent'):
        # keep index as new a column, will be restored and assigned back to X later
        idx = groupByOpponent.index

        # make match day an index because rolling need an index date
        xx = groupByOpponent.set_index('Date')
        xx['idx'] = idx
        # shift to exclude self
        xx['Last5AgainstThisOpponentWon'] = xx['Won'].rolling(6).apply(lambda x: np.nansum(x.shift()), raw=False)
        xx['Last5AgainstThisOpponentDraw'] = xx['Draw'].rolling(6).apply(lambda x: np.nansum(x.shift()), raw=False)

        xx['LastThisOpponentWon'] = xx['Won'].rolling(2).apply(lambda x: np.nansum(x.shift()), raw=False)
        xx['LastThisOpponentDraw'] = xx['Draw'].rolling(2).apply(lambda x: np.nansum(x.shift()), raw=False)

        # restore index
        xx = xx.set_index('idx')

        # assign back to the big dataframe
        X.loc[xx.index, 'Last5AgainstThisOpponentWon'] = xx['Last5AgainstThisOpponentWon']
        X.loc[xx.index, 'Last5AgainstThisOpponentDraw'] = xx['Last5AgainstThisOpponentDraw']
        
        X.loc[xx.index, 'LastThisOpponentWon'] = xx['LastThisOpponentWon']
        X.loc[xx.index, 'LastThisOpponentDraw'] = xx['LastThisOpponentDraw']

    # find recent forms
    idx = X.index
    xx = X.set_index('Date')
    xx['idx'] = idx
    xx['Last5Won'] = xx['Won'].rolling(6).apply(lambda x: np.nansum(x.shift()), raw=False)
    xx['Last5Draw'] = xx['Draw'].rolling(6).apply(lambda x: np.nansum(x.shift()), raw=False)
    xx['LastWon'] = xx['Won'].rolling(2).apply(lambda x: np.nansum(x.shift()), raw=False)
    xx['LastDraw'] = xx['Draw'].rolling(2).apply(lambda x: np.nansum(x.shift()), raw=False)
    
    # restore index
    xx = xx.set_index('idx')
    
    # assign back to the big dataframe
    X.loc[xx.index, 'Last5Won'] = xx['Last5Won']
    X.loc[xx.index, 'Last5Draw'] = xx['Last5Draw']
    X.loc[xx.index, 'LastWon'] = xx['LastWon']
    X.loc[xx.index, 'LastDraw'] = xx['LastDraw']

    X.loc[np.isnan(X['Last5AgainstThisOpponentWon']), 'Last5AgainstThisOpponentWon'] = 0
    X.loc[np.isnan(X['Last5AgainstThisOpponentDraw']), 'Last5AgainstThisOpponentDraw'] = 0
    X.loc[np.isnan(X['LastThisOpponentWon']), 'LastThisOpponentWon'] = 0
    X.loc[np.isnan(X['LastThisOpponentDraw']), 'LastThisOpponentDraw'] = 0
    X.loc[np.isnan(X['Last5Won']), 'Last5Won'] = 0
    X.loc[np.isnan(X['Last5Draw']), 'Last5Draw'] = 0
    X.loc[np.isnan(X['LastWon']), 'LastWon'] = 0
    X.loc[np.isnan(X['LastDraw']), 'LastDraw'] = 0

    # X = X.loc[X['Year'] >= 2011]
    Y = X[['Opponent', 'Result']]
    del X['Result']
    del X['Lost']
    del X['Draw']
    del X['Won']
    del X['FTR']
    del X['Date']

    # X_train = X[(X['Year'] >= 2011) & (X['Year'] <= 2016)]
    # Y_train = Y[(X['Year'] >= 2011) & (X['Year'] <= 2016)]
    X_train = X[(X['Year'] < predict_year)]
    Y_train = Y[(X['Year'] < predict_year)]
    X_test = X[(X['Year'] >= predict_year)]
    Y_test = Y[(X['Year'] >= predict_year)]

    # construct decision tree
    X_train_opponents = X_train.groupby('Opponent')
    Y_train_opponents = Y_train.groupby('Opponent')
    X_test_opponents = X_test.groupby('Opponent')
    Y_test_opponents = Y_test.groupby('Opponent')
    x_test_teams = X_test_opponents.groups.keys()
    X['Predict'] = ''
    os.makedirs(f'decision_tree/{league}/{predict_year}/{team}/', exist_ok=True)
    for key, X_train_opponent in X_train_opponents:
        if key not in x_test_teams:
            continue
        X_test_opponent = X_test_opponents.get_group(key)
        Y_train_opponent = Y_train_opponents.get_group(key)
        Y_test_opponent = Y_test_opponents.get_group(key)

        del Y_train_opponent['Opponent']
        del Y_test_opponent['Opponent']
        del X_train_opponent['Opponent']
        del X_test_opponent['Opponent']
        del X_train_opponent['Year']
        del X_test_opponent['Year']

        clf_entropy = DecisionTreeClassifier(
            # criterion="gini",
            criterion="entropy",
            random_state=100,
            # max_depth=3,
            min_samples_leaf=5
            # min_samples_leaf=3
        )
        clf_entropy.fit(X_train_opponent, Y_train_opponent)

        Y_pred = clf_entropy.predict(X_test_opponent)
        X.loc[X_test_opponent.index, 'Predict'] = Y_pred
        tree.export_graphviz(clf_entropy, out_file=f'decision_tree/{league}/{predict_year}/{team}/{key}.dot',
                             feature_names=X_test_opponent.columns.values,
                             class_names=clf_entropy.classes_)
        call(['dot', '-Tpng', f'decision_tree/{league}/{predict_year}/{team}/{key}.dot', '-o', f'decision_tree/{league}/{predict_year}/{team}/{key}.png'])

    X['Actual'] = Y['Result']
    x = X[X['Predict'] != '']
    x.to_csv(f'./decision_tree/{league}/{predict_year}/{team}/x.csv')
    print(f"{team} prediction accuracy is ", accuracy_score(x['Actual'], x['Predict'])*100)

    if df_league is None:
        df_league = x
    else:
        df_league = df_league.append(x, ignore_index=True, sort=False)
print("Overall accuracy is ", accuracy_score(df_league['Actual'], df_league['Predict'])*100)


Alaves prediction accuracy is  41.17647058823529
Ath Bilbao prediction accuracy is  27.77777777777778
Ath Madrid prediction accuracy is  55.55555555555556
Barcelona prediction accuracy is  63.888888888888886
Betis prediction accuracy is  30.555555555555557
Celta prediction accuracy is  33.33333333333333
Eibar prediction accuracy is  25.0
Espanol prediction accuracy is  36.11111111111111
Getafe prediction accuracy is  41.17647058823529
Girona prediction accuracy is  nan


/home/vachara/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:1128: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/vachara/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


La Coruna prediction accuracy is  36.11111111111111
Las Palmas prediction accuracy is  41.66666666666667
Leganes prediction accuracy is  34.375
Levante prediction accuracy is  31.25
Malaga prediction accuracy is  33.33333333333333
Real Madrid prediction accuracy is  61.111111111111114
Sevilla prediction accuracy is  44.44444444444444
Sociedad prediction accuracy is  25.0
Valencia prediction accuracy is  61.111111111111114
Villarreal prediction accuracy is  33.33333333333333
Overall accuracy is  39.88095238095239


In [8]:
df_league

,Year,HomeMatch,Opponent,Last5AgainstThisOpponentWon,Last5AgainstThisOpponentDraw,LastThisOpponentWon,LastThisOpponentDraw,Last5Won,Last5Draw,LastWon,LastDraw,Predict,Actual
0,2017,False,Leganes,0.0,0.0,0.0,1.0,3.0,2.0,0.0,1.0,Draw,Lose
1,2017,True,Barcelona,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,Lose,Lose
2,2017,False,Celta,0.0,0.0,1.0,0.0,2.0,1.0,0.0,0.0,Lose,Lose
3,2017,True,Villarreal,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,Win,Lose
4,2017,False,La Coruna,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,Win,Lose
5,2017,True,Real Madrid,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Lose,Lose
6,2017,True,Sociedad,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,Lose,Lose
7,2017,False,Betis,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,Win,Lose
8,2017,True,Valencia,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,Lose,Lose
9,2017,True,Espanol,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Draw,Win
